# 🏞 Convolutional Neural Network

In this notebook, we'll walk through the steps required to train your own convolutional neural network (CNN) on the CIFAR dataset

In [3]:
import numpy as np

from tensorflow.keras import layers, models, optimizers, utils, datasets
#from notebooks.utils import display

## 0. Parameters <a name="parameters"></a>

In [4]:
NUM_CLASSES = 10

## 1. Prepare the Data <a name="prepare"></a>

In [5]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [6]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = utils.to_categorical(y_train, NUM_CLASSES)
y_test = utils.to_categorical(y_test, NUM_CLASSES)

In [7]:
display(x_train[:10])
print(y_train[:10])

array([[[[0.23137255, 0.24313726, 0.24705882],
         [0.16862746, 0.18039216, 0.1764706 ],
         [0.19607843, 0.1882353 , 0.16862746],
         ...,
         [0.61960787, 0.5176471 , 0.42352942],
         [0.59607846, 0.49019608, 0.4       ],
         [0.5803922 , 0.4862745 , 0.40392157]],

        [[0.0627451 , 0.07843138, 0.07843138],
         [0.        , 0.        , 0.        ],
         [0.07058824, 0.03137255, 0.        ],
         ...,
         [0.48235294, 0.34509805, 0.21568628],
         [0.46666667, 0.3254902 , 0.19607843],
         [0.47843137, 0.34117648, 0.22352941]],

        [[0.09803922, 0.09411765, 0.08235294],
         [0.0627451 , 0.02745098, 0.        ],
         [0.19215687, 0.10588235, 0.03137255],
         ...,
         [0.4627451 , 0.32941177, 0.19607843],
         [0.47058824, 0.32941177, 0.19607843],
         [0.42745098, 0.28627452, 0.16470589]],

        ...,

        [[0.8156863 , 0.6666667 , 0.3764706 ],
         [0.7882353 , 0.6       , 0.13333334]

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


## 2. Build the model <a name="build"></a>

In [ ]:
input_layer = layers.Input((32, 32, 3))  # Define a camada de entrada com dimensões 32x32 e 3 canais de cor (RGB)

x = layers.Conv2D(
    filters=32,              # Número de filtros na camada de convolução
    kernel_size=3,           # Tamanho do kernel (3x3)
    strides=1,               # Passo da convolução (movimento do kernel). Se 1, o kernel se move um pixel por vez, em uma dimensão de 32x32, fica 31x31
    padding="same"           # Padding "same" mantém as dimensões da entrada
)(input_layer)
x = layers.BatchNormalization()(x)   # Normaliza as ativações para acelerar o treinamento
x = layers.LeakyReLU()(x)            # Função de ativação Leaky ReLU para introduzir não-linearidade

x = layers.Conv2D(
    filters=32,              # Número de filtros na segunda camada de convolução
    kernel_size=3,           # Tamanho do kernel (3x3)
    strides=2,               # Passo de 2 para reduzir as dimensões espaciais pela metade, ou seja, em uma dimensão de 32x32, fica 16x16
    padding="same"           # Padding "same" mantém as dimensões ajustadas após a convolução
)(x)
x = layers.BatchNormalization()(x)   # Normaliza as ativações
x = layers.LeakyReLU()(x)            # Função de ativação Leaky ReLU

x = layers.Conv2D(
    filters=64,              # Aumenta para 64 filtros na terceira camada de convolução
    kernel_size=3,           # Tamanho do kernel (3x3)
    strides=1,               # Passo de 1
    padding="same"           # Padding "same" mantém as dimensões
)(x)
x = layers.BatchNormalization()(x)   # Normaliza as ativações
x = layers.LeakyReLU()(x)            # Função de ativação Leaky ReLU

x = layers.Conv2D(
    filters=64,              # Mantém 64 filtros na quarta camada de convolução
    kernel_size=3,           # Tamanho do kernel (3x3)
    strides=2,               # Passo de 2 para reduzir novamente as dimensões espaciais
    padding="same"           # Padding "same" para manter as dimensões ajustadas
)(x)
x = layers.BatchNormalization()(x)   # Normaliza as ativações
x = layers.LeakyReLU()(x)            # Função de ativação Leaky ReLU

x = layers.Flatten()(x)              # Achata as ativações para conectar à camada densa

x = layers.Dense(128)(x)             # Camada densa com 128 neurônios
x = layers.BatchNormalization()(x)   # Normaliza as ativações
x = layers.LeakyReLU()(x)            # Função de ativação Leaky ReLU
x = layers.Dropout(rate=0.5)(x)      # Aplica dropout com taxa de 50% para evitar overfitting

x = layers.Dense(NUM_CLASSES)(x)     # Camada de saída com número de neurônios igual ao número de classes
output_layer = layers.Activation("softmax")(x)  # Ativação softmax para obtenção de probabilidades das classes

model = models.Model(input_layer, output_layer)    # Cria o modelo conectando a camada de entrada à camada de saída

model.summary()  # Exibe o resumo da arquitetura do modelo

## 3. Train the model <a name="train"></a>

In [ ]:
opt = optimizers.Adam(learning_rate=0.0005)
model.compile(
    loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
)

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    shuffle=True,
    validation_data=(x_test, y_test),
)

## 4. Evaluation <a name="evaluate"></a>

In [ ]:
model.evaluate(x_test, y_test, batch_size=1000)

In [ ]:
CLASSES = np.array(
    [
        "airplane",
        "automobile",
        "bird",
        "cat",
        "deer",
        "dog",
        "frog",
        "horse",
        "ship",
        "truck",
    ]
)

preds = model.predict(x_test)
preds_single = CLASSES[np.argmax(preds, axis=-1)]
actual_single = CLASSES[np.argmax(y_test, axis=-1)]

In [ ]:
import matplotlib.pyplot as plt

n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, idx in enumerate(indices):
    img = x_test[idx]
    ax = fig.add_subplot(1, n_to_show, i + 1)
    ax.axis("off")
    ax.text(
        0.5,
        -0.35,
        "pred = " + str(preds_single[idx]),
        fontsize=10,
        ha="center",
        transform=ax.transAxes,
    )
    ax.text(
        0.5,
        -0.7,
        "act = " + str(actual_single[idx]),
        fontsize=10,
        ha="center",
        transform=ax.transAxes,
    )
    ax.imshow(img)